Actualización de generación de cuentas de cobro

In [ ]:
!pip install reportlab
!pip install num2words
!pip install pdfkit
!pip install wkhtmltopdf


In [1]:
import pandas as pd
import pdfkit
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tkinter as tk
from tkinter import simpledialog
import pythoncom
import winshell
import time


Hacemos la función que pasa los valores en numeros para muestrarlos en letras
Diseñemos el formato de la cuenta de cobro de acuerdo al contenido
Se mejora el formato de visualización
Se guarda en pdf en una carpeta especifica con el nombre del artista
Seleccionamos el rango de las cuentas de cobro que queremos guardar

funcion para convertir numero a letras

In [8]:
def num_a_letras(numero):
    unidades = ['', 'Un', 'Dos', 'Tres', 'Cuatro', 'Cinco', 'Seis', 'Siete', 'Ocho', 'Nueve', 'Diez', 'Once', 'Doce', 'Trece', 'Catorce', 'Quince', 'Dieciséis', 'Diecisiete', 'Dieciocho', 'Diecinueve', 'Veinte', 'Veintiuno', 'Veintidos', 'Veintitres', 'Veinticuatro', 'Veinticinco', 'Veintiseis', 'Veintisiete', 'Veintiocho', 'Veintinueve']
    decenas = ['', 'Diez', 'Veinte', 'Treinta', 'Cuarenta', 'Cincuenta', 'Sesenta', 'Setenta', 'Ochenta', 'Noventa', 'Cien']
    centenas = ['', 'Ciento', 'Doscientos', 'Trescientos', 'Cuatrocientos', 'Quinientos', 'Seiscientos', 'Setecientos', 'Ochocientos', 'Novecientos']

    if numero == 0:
        return 'Cero'
    elif numero < 30:
        return unidades[numero]
    elif numero < 100:
        return decenas[numero // 10] + (' y ' + unidades[numero % 10] if numero % 10 != 0 else '')
    elif numero < 1000:
        return centenas[numero // 100] + (' ' + num_a_letras(numero % 100) if numero % 100 != 0 else '')
    elif numero < 2000:
        return 'Mil' + (' ' + num_a_letras(numero % 1000) if numero % 1000 != 0 else '')
    elif numero < 1000000:
        return num_a_letras(numero // 1000) + ' Mil' + (' ' + num_a_letras(numero % 1000) if numero % 1000 != 0 else '')
    elif numero < 2000000:
        return 'Un Millón' + (' ' + num_a_letras(numero % 1000000) if numero % 1000000 != 0 else '')
    elif numero < 2000000000:
        return num_a_letras(numero // 1000000) + ' Millones' + (' ' + num_a_letras(numero % 1000000) if numero % 1000000 != 0 else '')
    else:
        return 'ERROR: El número excede los límites.'

def convertir_num(numero, centimos_en_letra=True):
    moneda = "Peso"       # Nombre de Moneda (Singular)
    monedas = "Pesos"     # Nombre de Moneda (Plural)
    centimo = "Centavo"   # Nombre de Céntimos (Singular)
    centimos = "Centavos" # Nombre de Céntimos (Plural)
    preposicion = "Con"   # Preposición entre Moneda y Céntimos

    if 0 <= numero <= 1999999999.99:
        letra = num_a_letras(int(numero))

        if numero == 1:
            letra = letra + " " + moneda
        else:
            letra = letra + " " + monedas

        num_centimos = round((numero - int(numero)) * 100)

        if num_centimos >= 0:
            if centimos_en_letra:
                letra = letra + " " + preposicion + " " + num_a_letras(int(num_centimos))

                if num_centimos == 1:
                    letra = letra + " " + centimo
                else:
                    letra = letra + " " + centimos
            else:
                if num_centimos < 10:
                    letra = letra + " 0" + str(num_centimos) + "/100"
                else:
                    letra = letra + " " + str(num_centimos) + "/100"

        return letra
    else:
        return 'ERROR: El número excede los límites.'


diseño del pdf de cuenta de cobro con la información que queremos

In [12]:
import os
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def generar_carta_pdf(datos, cuentas_cobro=None):
    for index, row in datos.iterrows():
        cuenta_cobro = row['#Cuenta_cobro']

        # Verificar si se especificaron cuentas de cobro para procesar
        if cuentas_cobro is not None and cuenta_cobro not in cuentas_cobro:
            continue  # Saltar a la siguiente iteración si la cuenta de cobro no está en la lista

        nombre_artista = row['nombre_del_artista']
        identificacion = row['identificación']
        identificacion_con_formato = '{:,.0f}'.format(identificacion)  # Agregar formato de separador de miles
        valor = row['valor(numero)']
        concepto = row['concepto']
        nombre_evento = row['nombre_evento']
        fecha = row['fecha']
        cuenta_cobro = row['#Cuenta_cobro']
        empresa = row['empresa']

        # Crear el directorio "Cuentas" si no existe
        if not os.path.exists("E:/Aplicaciones/Bills/Cuentas"):
            os.mkdir("E:/Aplicaciones/Bills/Cuentas")

        # Ruta completa del PDF con el número de cuenta adelante
        pdf_file = f"E:/Aplicaciones/Bills/Cuentas/{cuenta_cobro}_{nombre_artista}_cuenta.pdf"
        c = canvas.Canvas(pdf_file, pagesize=letter)

        # Comprobar si el archivo ya existe
        counter = 1
        while os.path.exists(pdf_file):
            # Agregar sufijo al nombre del archivo
            pdf_file = f"E:/Aplicaciones/Bills/Cuentas/{cuenta_cobro}_{nombre_artista}_cuenta ({counter}).pdf"
            counter += 1

        # Establecer espacio entre líneas
        c.setFont("Helvetica", 12, leading=15)

        # Obtener el ancho de la página
        width, height = letter

        # Ajustar márgenes de 4 cm en los lados izquierdo y derecho (aprox. 113.4 puntos)
        left_margin = 113.4
        right_margin = letter[0] - 113.4

        # Ajustar márgenes de 3 cm en la parte superior e 2 cm en la parte inferior (aprox. 85.04 y 56.69 puntos)
        top_margin = letter[1] - 85.04
        bottom_margin = 56.69
        
         # Obtener el valor en letras
        valor_en_letras = convertir_num(valor)

        # Escribir el contenido de la carta con párrafos centrados y justificados dentro de los márgenes
        contenido = f"""Envigado, {fecha}

Cuenta de Cobro: {cuenta_cobro}
La empresa: {empresa}

DEBE A:
{nombre_artista}
Identificación: {identificacion_con_formato}  

El valor de: ($ {valor:,}) {valor_en_letras} pesos

Por Concepto de:
{concepto} por el evento: {nombre_evento}


"""

   # Dejar más espacio en blanco después del contenido sin centrar
        c.drawString(80, 300, "Cordialmente,")
        c.drawString(80, 145, f"{nombre_artista}")
        c.drawString(80, 120, f"{identificacion_con_formato}")
    
        # Dividir el contenido en líneas individuales
        lineas = contenido.split('\n')

        # Centrar y justificar cada línea dentro del ancho de la página
        y = top_margin
        for linea in lineas:
            x = c.drawCentredString(width / 2, y, linea)
            y -= 20  # Espacio entre líneas (ajustar según preferencia)

        c.save()

if __name__ == "__main__":
    # Leer el archivo de Excel
    df = pd.read_excel("E:/Aplicaciones/Bills/base_datos_artistas.xlsx", sheet_name="datos")

# Generar las cartas en PDF solo para las cuentas de cobro especificadas
    cuentas_especificas = [1, 3, 5]  # Agrega aquí las cuentas de cobro que deseas procesar
    generar_carta_pdf(df, cuentas_cobro=cuentas_especificas)

    # Generar las cartas en PDF
    #generar_carta_pdf(df)

In [32]:
import os
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import tkinter as tk
from tkinter import simpledialog
import pythoncom
import winshell
import time

def generar_carta_pdf(datos, cuenta_inicial=None, cuenta_final=None):
    for index, row in datos.iterrows():
        cuenta_cobro = row['#Cuenta_cobro']

        # Verificar si se especificó un rango de cuentas de cobro
        if cuenta_inicial is not None and cuenta_cobro < cuenta_inicial:
            continue  

        if cuenta_final is not None and cuenta_cobro > cuenta_final:
            break  # Salir del loop si el numero esta fuera del rango

        nombre_artista = row['nombre_del_artista']
        identificacion = row['identificación']
        identificacion_con_formato = '{:,.0f}'.format(identificacion)  # Agregar formato de separador de miles
        valor = row['valor(numero)']
        concepto = row['concepto']
        nombre_evento = row['nombre_evento']
        fecha = row['fecha']
        cuenta_cobro = row['#Cuenta_cobro']
        empresa = row['empresa']

        # Crear el directorio "Cuentas" si no existe
        if not os.path.exists("E:/Aplicaciones/Bills/Cuentas"):
            os.mkdir("E:/Aplicaciones/Bills/Cuentas")

        # Ruta completa del PDF con el número de cuenta adelante
        pdf_file = f"E:/Aplicaciones/Bills/Cuentas/{cuenta_cobro}_{nombre_artista}_cuenta.pdf"
        c = canvas.Canvas(pdf_file, pagesize=letter)

        # Comprobar si el archivo ya existe
        counter = 1
        while os.path.exists(pdf_file):
            # Agregar sufijo al nombre del archivo
            pdf_file = f"E:/Aplicaciones/Bills/Cuentas/{cuenta_cobro}_{nombre_artista}_cuenta ({counter}).pdf"
            counter += 1

        # Establecer espacio entre líneas
        c.setFont("Helvetica", 12, leading=15)

        # Obtener el ancho de la página
        width, height = letter

        # Ajustar márgenes de 4 cm en los lados izquierdo y derecho (aprox. 113.4 puntos)
        left_margin = 113.4
        right_margin = letter[0] - 113.4

        # Ajustar márgenes de 3 cm en la parte superior e 2 cm en la parte inferior (aprox. 85.04 y 56.69 puntos)
        top_margin = letter[1] - 85.04
        bottom_margin = 56.69
        
         # Obtener el valor en letras
        valor_en_letras = convertir_num(valor)

        # Escribir el contenido de la carta con párrafos centrados y justificados dentro de los márgenes
        contenido = f"""Envigado, {fecha}

Cuenta de Cobro: {cuenta_cobro}
La empresa: {empresa}

DEBE A:
{nombre_artista}
Identificación: {identificacion_con_formato}  

El valor de: ($ {valor:,}) {valor_en_letras} pesos

Por Concepto de:
{concepto} por el evento: {nombre_evento}


"""

   # Dejar más espacio en blanco después del contenido sin centrar
        c.drawString(80, 300, "Cordialmente,")
        c.drawString(80, 145, f"{nombre_artista}")
        c.drawString(80, 120, f"{identificacion_con_formato}")
    
        # Dividir el contenido en líneas individuales
        lineas = contenido.split('\n')

        # Centrar y justificar cada línea dentro del ancho de la página
        y = top_margin
        for linea in lineas:
            x = c.drawCentredString(width / 2, y, linea)
            y -= 20  # Espacio entre líneas (ajustar según preferencia)

        c.save()
        
# Function to show the GUI window and get the user input
def get_account_range():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    cuenta_inicial = simpledialog.askinteger("Cuenta Inicial", "Ingrese el número inicial de cuenta de cobro:")
    cuenta_final = simpledialog.askinteger("Cuenta Final", "Ingrese el número final de cuenta de cobro:")

    return cuenta_inicial, cuenta_final

# Function to create a shortcut on the desktop
def create_shortcut():
    desktop_path = winshell.desktop()
    shortcut_path = os.path.join(desktop_path, "Generar_Cartas.lnk")
    target_path = os.path.abspath(__file__)

    with pythoncom.CoCreateInstance(
        shell.CLSID_ShellLink, None,
        pythoncom.CLSCTX_INPROC_SERVER, shell.IID_IShellLink
    ) as shortcut:
        shortcut.SetPath(target_path)
        shortcut.SetDescription("Generar Cartas")
        with shortcut.QueryInterface(pythoncom.IID_IPersistFile) as shortcut_persist:
            shortcut_persist.Save(shortcut_path, 0)
            

if __name__ == "__main__":
    # Leer el archivo de Excel
    df = pd.read_excel("E:/Aplicaciones/Bills/base_datos_artistas.xlsx", sheet_name="datos")
# Preguntar al usuario el número inicial y final de cuenta de cobro

    cuenta_inicial, cuenta_final = get_account_range()
    
    if cuenta_inicial is not None or cuenta_final is not None:
        df = df[df['#Cuenta_cobro'].between(cuenta_inicial or 0, cuenta_final or 999999999)]
    
     # Generate the PDFs for the specified account number range
    generar_carta_pdf(df, cuenta_inicial=cuenta_inicial, cuenta_final=cuenta_final)
    
    
time.sleep(10)  # Cambia el valor de 10 a la cantidad de segundos que desees que la ventana permanezca abierta.

    # Create a shortcut on the desktop
    #create_shortcut()
    
    #C:\Users\Usuario\AppData\Local\Programs\Python\Python39 ubicación del intérprete de Python.
    #E:\Aplicaciones\Bills script de python cuenta de cobro
    
    # Ruta para el acceso directo "C:\Users\Usuario\AppData\Local\Programs\Python\Python39\python.exe" "E:\Aplicaciones\Bills\CuentasCobro.py"